In [1]:
!pip install geopandas pandas rasterstats rasterio

                                              0.0/1.1 MB ? eta -:--:--
     --------------------------------------   1.1/1.1 MB 33.0 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 23.1 MB/s eta 0:00:00
                                              0.0/22.7 MB ? eta -:--:--
     ---                                      2.2/22.7 MB 71.0 MB/s eta 0:00:01
     -------                                  4.3/22.7 MB 54.6 MB/s eta 0:00:01
     ----------                               5.9/22.7 MB 54.3 MB/s eta 0:00:01
     -------------                            7.5/22.7 MB 48.1 MB/s eta 0:00:01
     ----------------                         9.1/22.7 MB 44.6 MB/s eta 0:00:01
     ------------------                      10.6/22.7 MB 43.7 MB/s eta 0:00:01
     --------------------                    12.1/22.7 MB 40.9 MB/s eta 0:00:01
     ---------------------                   12.7/22.7 MB 40.9 MB/s eta 0:00:01
     ----------------------                  13.2/22.7 MB 3

In [8]:
# script que extrair mediana de poligono buffer da imagem.

import numpy as np
import rasterio
from rasterstats import zonal_stats
import os
import geopandas as gpd
import pandas as pd

# define o valor de nodata como nan
nodata = None 

# caminho shape transformado em geodataframe
estacoes = gpd.read_file("D:/mestrado/dissertacao/dados/vetores/Amz/estacoes_buffer_flags2.shp")

# caminho da pasta superior
root_dir = "E:/rasters/Amz/DSF_glint"

# criar um dataframe vazio para armazenar os dados
df = pd.DataFrame(columns=['campanha_i','estacoes_i','flag','flag_type', 'lat', 'long', 'date1', 'time', 'band_1_median', 'band_2_median', 'band_3_median', 'band_4_median', 'band_5_median', 'band_6_median', 'band_7_median', 'band_8_median', 'img_id'])

# percorrer as pastas na pasta superior
for dirpath, dirnames, filenames in os.walk(root_dir):
    # verificar se há arquivos TIFF na pasta atual
    tiff_files = [f for f in filenames if f.endswith("_merge.tif")]
    if not tiff_files:
        continue
    
    # percorrer os arquivos TIFF na pasta atual
    for tiff_file in tiff_files:
        # construir o caminho completo para o arquivo TIFF
        path_to_file = os.path.join(dirpath, tiff_file)

        # extrair os primeiros 18 caracteres do nome do arquivo TIFF
        img_id = os.path.basename(path_to_file)[:18]
        
        # percorrer as linhas do shapefile
        for index, row in estacoes.iterrows():
            # extrair a data da linha atual do shapefile
            date1 = row["date1"]

            # verificar se a data da imagem é igual à data na tabela de atributos
            img_date = os.path.splitext(os.path.basename(path_to_file))[0][:8]
            if img_date != date1:
                print(f"A data da imagem ({img_date}) é diferente da data na tabela de atributos ({date1})")
                continue

            # abrir o arquivo TIFF com rasterio
            with rasterio.open(path_to_file) as src:
                affine = src.transform
                band_medians = []
                band_dict = {}

                # adicionar os valores ao dataframe
                new_row = {'estacoes_i': row["estacoes_i"],
                           'campanha_i': row["campanha_i"],
                           'flag': row["flag"],
                           'flag_type': row["flag_type"],
                           'lat': row["lat"],
                           'long': row["long"],
                           'date1': date1,
                           'time': row["time"],
                           'img_id': img_id}

                for band in range(1, src.count + 1):
                    # extrair a geometria da estação da linha atual do shapefile
                    geom = row["geometry"]
                    stats = zonal_stats(geom, src.read(band), affine=affine, nodata=nodata, stats="count median")

                    # imprimir valores para cada banda e estação
                    print(f"band {band}: {stats}")

                    # adicionar os valores ao dicionário
                    band_dict[f'band_{band}_median'] = stats[0]["median"] if stats[0]["count"] != 0 else nodata
                    
                

                new_row.update(band_dict)
                df = pd.concat([df, pd.DataFrame(new_row, index=[0])], ignore_index=True)
                
                
# salvar o dataframe em um arquivo CSV
df.to_csv('D:/mestrado/dissertacao/dados/tabelas/resultados/Amz1/DSF_glint_stats.csv', index=False)

A data da imagem (20210902) é diferente da data na tabela de atributos (20210831)
A data da imagem (20210902) é diferente da data na tabela de atributos (20210831)
A data da imagem (20210902) é diferente da data na tabela de atributos (20210831)
A data da imagem (20210902) é diferente da data na tabela de atributos (20210831)
A data da imagem (20210902) é diferente da data na tabela de atributos (20210901)
A data da imagem (20210902) é diferente da data na tabela de atributos (20210901)
A data da imagem (20210902) é diferente da data na tabela de atributos (20210901)
A data da imagem (20210902) é diferente da data na tabela de atributos (20210901)
band 1: [{'count': 32, 'median': 0.015224973671138287}]
band 2: [{'count': 32, 'median': 0.017810270190238953}]
band 3: [{'count': 32, 'median': 0.029105842113494873}]
band 4: [{'count': 32, 'median': 0.038484200835227966}]
band 5: [{'count': 32, 'median': 0.043378666043281555}]
band 6: [{'count': 32, 'median': 0.04508057236671448}]
band 7: [

band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 35, 'median': 0.0023102648556232452}]
band 2: [{'count': 35, 'median': 0.0004231444909237325}]
band 3: [{'count': 35, 'median': 0.0034512935671955347}]
band 4: [{'count': 35, 'median': 0.004849129822105169}]
band 5: [{'count': 35, 'median': 0.00022598910436499864}]
band 6: [{'count': 35, 'median': -0.004009815398603678}]
band 7: [{'count': 35, 'median': -

band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'me

band 1: [{'count': 35, 'median': 0.06582781672477722}]
band 2: [{'count': 35, 'median': 0.09165729582309723}]
band 3: [{'count': 35, 'median': 0.12101687490940094}]
band 4: [{'count': 35, 'median': 0.11790187656879425}]
band 5: [{'count': 35, 'median': 0.13865679502487183}]
band 6: [{'count': 35, 'median': 0.1413632333278656}]
band 7: [{'count': 35, 'median': 0.12712861597537994}]
band 8: [{'count': 35, 'median': 0.09852661192417145}]
band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 34, 'median': 0.07731941342353821}]
band 2: [{'count': 34, 'median': 0.09710383415222168}]
band 3: [{'count': 34, 'median': 0.12749752402305603}]
band 4: [{'count': 34, 'median': 0.13178399205207825}]
band 5: [{'count': 34, 'media

band 1: [{'count': 32, 'median': -0.012152397073805332}]
band 2: [{'count': 32, 'median': -0.006727396510541439}]
band 3: [{'count': 32, 'median': -0.0007626592414453626}]
band 4: [{'count': 32, 'median': 0.0025456990115344524}]
band 5: [{'count': 32, 'median': -0.003123082686215639}]
band 6: [{'count': 32, 'median': -0.009337764233350754}]
band 7: [{'count': 32, 'median': -0.0057463860139250755}]
band 8: [{'count': 32, 'median': 0.0}]
band 1: [{'count': 34, 'median': 0.0018195942975580692}]
band 2: [{'count': 34, 'median': 0.011671148240566254}]
band 3: [{'count': 34, 'median': 0.01808454841375351}]
band 4: [{'count': 34, 'median': 0.025529984384775162}]
band 5: [{'count': 34, 'median': 0.025659486651420593}]
band 6: [{'count': 34, 'median': 0.0207795649766922}]
band 7: [{'count': 34, 'median': 0.015642527490854263}]
band 8: [{'count': 34, 'median': 0.0}]
band 1: [{'count': 36, 'median': 0.0016640182584524155}]
band 2: [{'count': 36, 'median': 0.009194518439471722}]
band 3: [{'count':

...

band 5: [{'count': 32, 'median': 0.14300478994846344}]
band 6: [{'count': 32, 'median': 0.13415054976940155}]
band 7: [{'count': 32, 'median': 0.12880146503448486}]
band 8: [{'count': 32, 'median': 0.09469059109687805}]
band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
A data da imagem (20220320) é diferente da data na tabela de atributos (20220321)
A data da imagem (20220320) é diferente da data na tabela de atributos (20

band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 34, 'median': 0.0203409381210804}]
band 2: [{'count': 34, 'median': 0.03369494527578354}]
band 3: [{'count': 34, 'median': 0.05657058581709862}]
band 4: [{'count': 34, 'median': 0.07961146533489227}]
band 5: [{'count': 34, 'median': 0.08662718534469604}]
band 6: [{'count': 34, 'median': 0.09415409713983536}]
band 7: [{'count': 34, 'median': 0.077788114547

band 1: [{'count': 35, 'median': 0.03508824110031128}]
band 2: [{'count': 35, 'median': 0.041296400129795074}]
band 3: [{'count': 35, 'median': 0.06302055716514587}]
band 4: [{'count': 35, 'median': 0.07591904699802399}]
band 5: [{'count': 35, 'median': 0.07180534303188324}]
band 6: [{'count': 35, 'median': 0.06913679093122482}]
band 7: [{'count': 35, 'median': 0.09736865758895874}]
band 8: [{'count': 35, 'median': 0.14765460789203644}]
band 1: [{'count': 34, 'median': 0.014775024726986885}]
band 2: [{'count': 34, 'median': 0.016852088272571564}]
band 3: [{'count': 34, 'median': 0.031403906643390656}]
band 4: [{'count': 34, 'median': 0.044424284249544144}]
band 5: [{'count': 34, 'median': 0.039011090993881226}]
band 6: [{'count': 34, 'median': 0.042659640312194824}]
band 7: [{'count': 34, 'median': 0.03394591808319092}]
band 8: [{'count': 34, 'median': 0.0}]
band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'c

band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 0, 'median': None}]
band 2: [{'count': 0, 'median': None}]
band 3: [{'count': 0, 'median': None}]
band 4: [{'count': 0, 'median': None}]
band 5: [{'count': 0, 'median': None}]
band 6: [{'count': 0, 'median': None}]
band 7: [{'count': 0, 'median': None}]
band 8: [{'count': 0, 'median': None}]
band 1: [{'count': 34, 'median': 0.020293792709708214}]
band 2: [{'count': 34, 'median': 0.03445950150489807}]
band 3: [{'count': 34, 'median': 0.05514076352119446}]
band 4: [{'count': 34, 'median': 0.08135689049959183}]
band 5: [{'count': 34, 'median': 0.09725484251976013}]
band 6: [{'count': 34, 'median': 0.09689183533191681}]
band 7: [{'count': 34, 'median': 0.08331072330474854}]
band 8: [{'count': 34, 'media

band 1: [{'count': 35, 'median': 0.031740158796310425}]
band 2: [{'count': 35, 'median': 0.030292456969618797}]
band 3: [{'count': 35, 'median': 0.05630483105778694}]
band 4: [{'count': 35, 'median': 0.0702843889594078}]
band 5: [{'count': 35, 'median': 0.06524979323148727}]
band 6: [{'count': 35, 'median': 0.055477455258369446}]
band 7: [{'count': 35, 'median': 0.09097017347812653}]
band 8: [{'count': 35, 'median': 0.14906196296215057}]
band 1: [{'count': 34, 'median': 0.020928677171468735}]
band 2: [{'count': 34, 'median': 0.019822610542178154}]
band 3: [{'count': 34, 'median': 0.033446069806814194}]
band 4: [{'count': 34, 'median': 0.0502927303314209}]
band 5: [{'count': 34, 'median': 0.05322832614183426}]
band 6: [{'count': 34, 'median': 0.04454521834850311}]
band 7: [{'count': 34, 'median': 0.04119846224784851}]
band 8: [{'count': 34, 'median': 0.0}]
band 1: [{'count': 34, 'median': 0.030582191422581673}]
band 2: [{'count': 34, 'median': 0.040409646928310394}]
band 3: [{'count': 3